# Redis Basics Tutorial

Introduction aux bases de Redis avec notre dataset de films.

In [44]:
import sys
import os
from pprint import pprint 

sys.path.append('/home/jovyan/work')

from utils.redis_client import RedisClient

# Initialiser la connexion
r = RedisClient(host='redis')

# Tester la connexion
print("Connection test:", r.ping())

# Récupérer un film
movie = r.get_movie(1)
print("\nMovie details:", movie)

Connection test: True

Movie details: {'plot': 'A group of intergalactic criminals must pull together to stop a fanatical warrior with plans to purge the universe.', 'poster': 'https://m.media-amazon.com/images/M/MV5BMTAwMjU5OTgxNjZeQTJeQWpwZ15BbWU4MDUxNDYxODEx._V1_SX300.jpg', 'release_year': '2014', 'genre': 'Action', 'rating': '8.1', 'votes': '704613', 'ibmdb_id': 'tt2015381', 'title': 'New Title'}


In [36]:
import redis

In [37]:
# Pour connaître le type des clés dans la base de données 
key_type = r.client.type('user:2572')  # Vérifie le type de la clé
print("Type of 'user:2572':", key_type)


Type of 'user:2572': hash


In [38]:
# Selon le type de la clé on choisit la bonne commande 
#Pour un Hash, utilisez HGET ou HGETALL.
#Pour une Liste, utilisez LRANGE ou LPOP.
#Pour un Set, utilisez SMEMBERS.
#Pour un Sorted Set, utilisez ZRANGE. 
from pprint import pprint 


# Récupère tous les champs du Hash
user_data = r.client.hgetall('user:2572')  
pprint("User Data for 'user:2572':", )
pprint({k: v for k, v in user_data.items()})

#Récupère le champ 'title' du Hash
#r.client.hget('movie:1', 'title')
r.client.hget('user:2572', 'email')

"User Data for 'user:2572':"
{'city': 'Hwacheon',
 'country': 'South-Korea',
 'country_code': 'KR',
 'email': 'dgallopfv@yelp.com',
 'first_name': 'Dew',
 'gender': 'Male',
 'ip_address': '41.121.156.99',
 'last_login': '1588788451',
 'last_name': 'Gallop',
 'location': '127.7080492,38.1056484'}


'dgallopfv@yelp.com'

In [39]:
r.client.hmget('movie:1', ['title', 'genre'])

['New Title', 'Action']

In [ ]:
# Vérifier si une clé existe
exists = r.client.exists('user:2572')
#print(exists)
if exists == 1 :
    print("Does 'user:2572' exist?", "true")
else :
    print("Does 'user:2572' exist?", "false")

r.client.hset('movie:1', 'title', 'New Title')

Does 'user:2572' exist? true


0

In [41]:
r.client.hmget('movie:1', ['title', 'genre'])

['New Title', 'Action']

In [69]:
# Utiliser un pipeline pour exécuter plusieurs commandes en même temps

pipe = r.client.pipeline()

pipe.hset('user:2573', "name", "Alice")
pipe.hset('user:2573', "age", 40)
pipe.hincrby('user:2573', "login_count", 1)
results = pipe.execute()

print(results)  
r.client.hmget('user:2573', ['name', 'age'])


[1, 1, 1]


['Alice', '40']

In [ ]:
### Expiration d'une clé 

import time 

# Ajouter des données
r.client.hset("user:2001", mapping={"name": "Bob", "age": "25", "city": "Lyon"})

r.client.expire("user:2001", 10)

print("Avant expiration:", r.client.hgetall("user:2001"))

time.sleep(12)

print("Après expiration:", r.client.hgetall("user:2001"))


Avant expiration: {'name': 'Bob', 'age': '25', 'city': 'Lyon'}
Après expiration: {}


In [76]:
### Suppression de données 

r.client.hset("user:2002", mapping={"name": "Charles ", "age": "28", "city": "Marseille"})

print("Avant suppression:")
pprint(r.client.hgetall("user:2002"))
r.client.hdel('user:2002', 'age')

print('après suppression du champ age:')
pprint(r.client.hgetall("user:2002"))

r.client.delete("user:2002")

print("Après suppression du user:")
pprint(r.client.hgetall("user:2002"))  

Avant suppression:
{'age': '28', 'city': 'Marseille', 'name': 'Charles '}
après suppression du champ age:
{'city': 'Marseille', 'name': 'Charles '}
Après suppression du user:
{}


In [ ]:
keys = list(r.client.scan_iter())
print (keys)

prefixes = {key.split(":")[0] for key in keys}

print("Préfixes uniques:", prefixes)

['user:5200', 'user:4795', 'user:5063', 'user:4789', 'user:2747', 'user:3074', 'user:2824', 'user:1891', 'user:4596', 'user:5899', 'user:439', 'user:650', 'movie:149', 'user:4231', 'user:1204', 'actor:1217', 'user:4169', 'movie:1096', 'movie:26', 'actor:988', 'user:3433', 'user:3719', 'user:993', 'movie:219', 'actor:765', 'user:2959', 'user:1180', 'movie:180', 'user:3645', 'user:4568', 'user:4105', 'user:645', 'user:3965', 'user:1175', 'user:949', 'user:1751', 'user:5802', 'user:3545', 'user:5071', 'user:5918', 'user:4539', 'user:5638', 'user:5931', 'user:5196', 'user:3895', 'user:5746', 'user:4938', 'user:1202', 'user:5219', 'user:3899', 'user:694', 'movie:825', 'actor:1254', 'user:5094', 'movie:763', 'actor:79', 'actor:474', 'actor:1230', 'actor:538', 'actor:229', 'movie:111', 'user:4614', 'user:997', 'user:3392', 'movie:751', 'user:2921', 'user:4774', 'user:1277', 'user:4014', 'user:5311', 'user:820', 'user:3722', 'user:2955', 'user:1383', 'user:4036', 'user:818', 'actor:560', 'user